# 📅 QEPC Daily Predictions  
This notebook runs QEPC predictions for **today's NBA games** using your  
rolling-strength engine and Poisson simulation model.

Outputs include:

- Home win probability  
- Predicted margin  
- Predicted total points  
- QEPC strength breakdown  
- (Optional) Player projections & recommended SGP legs  


In [ ]:
import sys
sys.path.append("..")
sys.path.append("../qepc_project")

import pandas as pd
import numpy as np
import datetime

# QEPC imports
from qepc_project.qepc.utils.loader import load_games, load_team_stats
from qepc_project.qepc.engine.strengths import build_team_strengths
from qepc_project.qepc.engine.matchup import compute_matchup_factors
from qepc_project.qepc.models.nba import run_nba_simulation

print("✅ QEPC Daily Prediction Notebook Ready.")


## 📂 Load Historical Data  
We load the season's Games.csv + TeamStatistics.csv so QEPC can calculate rolling strengths.


In [ ]:
games = load_games('../data/Games.csv')
team_stats = load_team_stats('../data/TeamStatistics.csv')

print("Games:", len(games))
print("Team Logs:", len(team_stats))


## 🔧 Build Rolling Strength Engine  
This computes each team's offensive factor, defensive factor, and pace factor  
using exponential decay.


In [ ]:
team_strengths = build_team_strengths(team_stats)
print("Teams calculated:", len(team_strengths))
list(team_strengths.keys())[:10]


# 📅 Identify Today's Games  
We filter the schedule to games occurring **today** based on system date.


In [ ]:
games['gameDate'] = pd.to_datetime(games['gameDate'])

today = pd.Timestamp.now().date()
print("Today's Date:", today)

todays_games = games[games['gameDate'].dt.date == today]

if todays_games.empty:
    print("No NBA games today.")
else:
    todays_games


# 🔮 QEPC Predictions for Today's Games  
For each matchup, QEPC generates:

- Home win probability  
- Projected margin  
- Projected total points  
- Strength factors  


In [ ]:
predictions = []

for _, game in todays_games.iterrows():
    home = game['hometeamName']
    away = game['awayteamName']

    # Compute matchup factors
    home_str, away_str, pace = compute_matchup_factors(home, away, team_strengths)

    # Run Poisson universe simulations
    sim = run_nba_simulation(
        home=home,
        away=away,
        home_strength=home_str,
        away_strength=away_str,
        pace=pace
    )

    predictions.append({
        "Away": away,
        "Home": home,
        "Home Win %": round(sim.home_win_prob * 100, 1),
        "Pred Margin": round(sim.avg_margin, 2),
        "Pred Total": round(sim.total_mean, 1),
        "Home Strength": round(home_str, 3),
        "Away Strength": round(away_str, 3),
        "Pace Factor": round(pace, 3)
    })

pd.DataFrame(predictions)


## 📊 Visual Overview  
Scatter plot of predicted margin vs predicted total.


In [ ]:
if predictions:
    pdf = pd.DataFrame(predictions)

    plt.figure(figsize=(10,6))
    plt.scatter(pdf['Pred Margin'], pdf['Pred Total'], s=120)

    for _, row in pdf.iterrows():
        plt.text(row['Pred Margin'], row['Pred Total'], row['Home'], fontsize=9)

    plt.title("QEPC — Today's Predicted Margins & Totals")
    plt.xlabel("Predicted Margin (Home - Away)")
    plt.ylabel("Predicted Total Points")
    plt.grid(alpha=0.3)
    plt.show()


## 💾 Export Predictions  
Save today's predictions for logging, betting tools, or future analysis.


In [ ]:
output_path = f"../data/QEPC_daily_predictions_{today}.csv"
pd.DataFrame(predictions).to_csv(output_path, index=False)

print(f"Saved to: {output_path}")


# 🏁 QEPC Daily Predictions Ready  
This notebook now gives you:

- Real-time predictions  
- Rolling strength-based simulation  
- Exportable reports  
- Clean visuals  

Next steps:
- Integrate Vegas odds  
- Add injury adjustments  
- Add player projections  
- Build SGP generator  
